In [32]:
import json
from gptgen import *
from tqdm import trange
import numpy as np
from copy import deepcopy

import pandas as pd

In [2]:
gpt_completion("hello World", 12)

['hello World" will be displayed.\n\nThe "hello" command',
 'hello World" will be rendered.\n\nTo find out more about',
 'hello World!\n\nYou are in the middle of a test,',
 'hello World\n\nThe most important thing to note is that there are',
 'hello World")); // => "Hello World"\n\n// A',
 'hello World!" ); }\n\nI like how the language supports this',
 'hello World", "hello World".println); // Hello world. //',
 'hello World", "Hello World!", "Hello World! ", ""',
 'hello World" message on the phone. But it could happen with any',
 'hello World>\n\nimport std.stdio; void main()']

In [3]:
with open("../data/nth_letter.json", "r") as f:
  nth_dataset = json.load(f)

In [4]:
nth_dataset.keys()

dict_keys(['questions', 'samples'])

In [5]:
def get_sample(question, examples, questions):
  t = f"{question}\n"
  for x in examples:
    t = t + f"{x}\n"

  prompts = []
  for q in questions:
    prompts.append(t + q[:-2])

  return prompts

def run_and_store_result(p):
  corr = 0
  for x in gpt_completion(p, 1):
    qline = x.split("\n")[len(p.split("\n")) - 1]
    in_, out_ = qline.split("->")[1:]
    if in_.strip()[0] == out_.strip():
      corr += 1
  return corr/10

In [65]:
N_SAMPLE = 50
results = []
task_names = ["hippo-banana", "night-owl", "pink-toe", "carol-shelby"]
for n in range(4, 8, 1):
  samples = np.array(nth_dataset["samples"][n])
  for ex_size in range(1, 15, 4):
    idx = [i for i in range(len(samples))]
    np.random.shuffle(idx)
    train_idx = idx[:ex_size]
    test_idx = idx[ex_size: ex_size + N_SAMPLE]
    
    # train_string
    train_string = f"What is the {n}th letter in each word?\n" + "\n".join(samples[train_idx])
    test_strings = samples[test_idx].tolist()

    for temp in [0.8, 0.9]:
      for topk in [10, 30]:
        for topp in [0.8, 0.99]:
          for nbeams in [1, 5, 10]:
            results.append({
              "accuracy": np.random.random(len(test_idx)).tolist(),
              "train_string": train_string,
              "test_strings": test_strings,
              "task_id": task_names[n-4],
              "number_of_examples": ex_size,
              "temprature": temp,
              "top_k": topk,
              "top_p": topp,
              "number_of_beams": nbeams,
            })

In [66]:
print(train_string)

What is the 7th letter in each word?
in -> shopping; out -> g
in -> current; out -> NA
in -> related; out -> NA
in -> percent; out -> NA
in -> stories; out -> NA
in -> possible; out -> e
in -> website; out -> NA
in -> usually; out -> NA
in -> services; out -> s
in -> following; out -> n
in -> requirements; out -> m
in -> security; out -> y
in -> already; out -> NA


In [71]:
with open("./dummy_res.json", "w") as f:
  f.write(json.dumps(results))

In [68]:
df = pd.DataFrame(results)

In [70]:
print(df.sample(10))

                                              accuracy  \
255  [0.8824503376688111, 0.6170729089688675, 0.104...   
276  [0.1536049340187916, 0.15704183779111525, 0.54...   
347  [0.6083017760437126, 0.8493251023010111, 0.521...   
189  [0.545747498215702, 0.9450325425993152, 0.6863...   
319  [0.9930589852899564, 0.539873708801895, 0.3394...   
316  [0.6779940463470899, 0.9769774584569012, 0.849...   
250  [0.05265993704211969, 0.6220768777625025, 0.18...   
77   [0.12936407392327087, 0.38335917366573313, 0.6...   
79   [0.061912034717582, 0.6293456749416316, 0.1603...   
104  [0.17092878120229527, 0.6440747161034692, 0.08...   

                                          train_string  \
255  What is the 6th letter in each word?\nin -> ti...   
276  What is the 6th letter in each word?\nin -> ac...   
347  What is the 7th letter in each word?\nin -> ma...   
189  What is the 5th letter in each word?\nin -> pr...   
319  What is the 7th letter in each word?\nin -> pr...   
316  What is 

In [47]:
results[0]["task"]

KeyError: 'task'

In [6]:
results = {}
ex_size = 10
for n in range(1):
  # gather samples for all questions
  
  

  this_n_res = {"ex_idx": ex_idx, "res": []}
  
  for qidx in range(1):
    prompts = get_sample(
      question = nth_dataset["questions"][n][qidx],
      examples = samples[ex_idx],
      questions = samples[test_idx]
    )
    
    res = []
    for i, p in zip(trange(len(prompts)), prompts):
      res.append(run_and_store_result(p))
      
    this_n_res["res"].append(res)
    
  results[n] = this_n_res

100%|██████████| 30/30 [00:30<00:00,  1.01s/it]


In [7]:
results

{0: {'ex_idx': array([ 34,   0, 194, 140, 180, 146,  70, 193,  39, 145]),
  'res': [[0.3,
    0.1,
    0.6,
    0.3,
    0.4,
    0.7,
    0.9,
    1.0,
    0.3,
    0.6,
    0.3,
    0.5,
    0.8,
    0.3,
    0.2,
    0.1,
    0.2,
    0.5,
    0.3,
    0.7,
    0.1,
    0.6,
    0.5,
    0.3,
    0.9,
    0.3,
    0.6,
    0.1,
    0.2,
    0.8]]}}

In [74]:
QIDX = 1


In [77]:
print(prompts[9])

Output letter at 0th location
in -> subject; out -> s
in -> document; out -> d
in -> science; out -> s
in -> projects; out -> p
in -> provides; out -> p
in -> international; out -> i
in -> contact; out -> c
in -> september; out -> s
in -> english; out -> e
in -> medical; out -> m
in -> message; out ->


100%|██████████| 227/227 [03:50<00:00,  1.02s/it]


In [79]:
res

[0.5,
 0.7,
 0.3,
 0.8,
 1.0,
 0.9,
 0.3,
 0.4,
 0.4,
 0.4,
 0.9,
 0.2,
 0.5,
 0.3,
 0.8,
 0.6,
 0.3,
 0.0,
 0.6,
 0.8,
 0.8,
 0.5,
 0.6,
 0.2,
 0.6,
 0.5,
 0.5,
 0.9,
 0.5,
 0.4,
 0.4,
 0.8,
 0.2,
 0.4,
 0.8,
 0.2,
 0.7,
 0.8,
 0.6,
 0.8,
 0.5,
 0.9,
 0.6,
 0.4,
 0.9,
 0.7,
 0.7,
 0.2,
 0.5,
 0.2,
 0.9,
 0.2,
 0.5,
 0.3,
 1.0,
 0.9,
 0.2,
 0.7,
 0.6,
 0.5,
 0.3,
 0.4,
 0.7,
 0.3,
 0.9,
 0.4,
 0.4,
 0.5,
 0.5,
 0.4,
 0.3,
 0.8,
 0.3,
 0.9,
 0.3,
 0.3,
 0.7,
 0.6,
 0.1,
 0.6,
 0.4,
 0.5,
 0.8,
 0.8,
 0.4,
 0.8,
 0.6,
 0.5,
 0.3,
 0.3,
 0.7,
 0.6,
 0.2,
 0.9,
 0.0,
 0.1,
 0.6,
 0.9,
 0.7,
 0.5,
 0.4,
 0.7,
 0.3,
 0.3,
 0.3,
 0.8,
 0.3,
 0.4,
 0.4,
 0.5,
 0.5,
 0.5,
 0.2,
 0.9,
 0.9,
 0.2,
 0.7,
 0.9,
 1.0,
 0.6,
 1.0,
 0.3,
 0.2,
 0.6,
 0.2,
 0.3,
 0.2,
 0.6,
 0.9,
 0.5,
 0.4,
 0.2,
 0.8,
 0.0,
 0.7,
 0.6,
 0.6,
 0.9,
 0.7,
 0.9,
 0.3,
 0.8,
 0.5,
 0.1,
 0.4,
 0.2,
 0.6,
 0.3,
 0.5,
 0.7,
 0.1,
 0.7,
 0.5,
 0.7,
 0.6,
 0.6,
 0.4,
 0.5,
 0.2,
 0.4,
 0.9,
 0.6,
 0.3,
 0.2,
 0.5,
 0.1,
 0.4

In [73]:
r0 = deepcopy(res)

In [76]:
nth_dataset["questions"][NTH][0]

'What is the letter at 0th location?'

In [ ]:
results = {}